# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


In [1]:
"""
Taylor

Definición de polinomios de Taylor, donde v representa los coeficientes 
\$f_{[k]} = \\frac{f^{(k)}(x_0)}{k!} = \\frac{1}{k!}\\frac{d^k f}{dx^k}(x_0)\$ evaluados en \$x_0\$ y n es 
el grado del polinomio.
"""
type Taylor{T<:Number}
    v::Array{T,1}
    n::Int
end
function Taylor{T<:Number}(v::T)#función Taylor aplicada sobre un número
    x=[zero(eltype(v))]
    x[1]=v
    return Taylor(x,0)#devuelve una estructura Taylor de orden 0
end
function Taylor{T<:Number}(v::Array{T,1})#función Taylor aplicada sobre un vector
    i=length(v)#número de elementos del vector v
    while(v[i]==0&& i>1)
        i=i-1
    end
    x=zeros(eltype(v), i)#vector con i elementos
    for j in 1:i
        x[j]=v[j]
    end
    return Taylor(x,i-1)#regresa un Taylor con el orden mayor del vector v
end
"""
taylor(v,n)
Función que representa una estructura Taylor con n+1 elementos de v. Si los elementos de v son menores 
a n llena los elementos faltantes con ceros. Siempre regresa el orden del vector v.
"""
function taylor(v,n)#función Taylor aplicada sobre un vector
    x=Taylor(v)
    a=zeros(eltype(v), n+1)
    if x.n==n
        return x
    elseif x.n<n
        for i in 1:x.n+1
            a[i]=x.v[i]
        end
        return Taylor(a,x.n)
    else
        for i in 1:n+1
            a[i]=x.v[i]
        end
        return Taylor(a,x.n)
    end
end

taylor

**Operaciones aritméticas para coeficientes de Taylor**
\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

In [2]:
"""
taylorsum(a,b)
Función que suma dos estructuras Taylor a y b.
"""
function taylorsum(a,b)#función que suma los coeficientes tipo Taylor
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.n==y.n#comparo los ordenes de la estructura Taylor de a y b
        return Taylor(x.v+y.v)#suma los vectores y regresa un Taylor
    elseif x.n<y.n
        z=taylor(x.v,y.n)
        return Taylor(z.v+y.v)
    else 
        z=taylor(y.v,x.n)
        return Taylor(x.v+z.v)
    end
end
"""
taylorres(a,b)
Función que resta dos estructuras Taylor a y b.
"""
function taylorres(a,b)#función que suma los coeficientes tipo Taylor
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.n==y.n#comparo los ordenes de la estructura Taylor de a y b
        return Taylor(x.v-y.v)#resta los vectores y regresa un Taylor
    elseif x.n<y.n
        z=taylor(x.v,y.n)
        return Taylor(z.v-y.v)
    else 
        z=taylor(y.v,x.n)
        return Taylor(x.v-z.v)
    end
end
"""
producto(a,b)
Función que regresa el producto de dos vectores a,b de igual magnitud. Implementa las operaciones aritméticas de 
Taylor.
"""
function producto(a,b)#producto de los coeficientes tipo Taylor para dos vectores de igual número de elementos
    ab=zeros(eltype(a), length(a))#vector con los mismos elementos de a
    for k in 1:length(a)
        for i in 1:k
            ab[k]+=a[i]*b[k+1-i]
        end
    end
    return ab
end
"""
taylorprod(a,b)
Función que regresa el producto de dos estructuras tipo Taylor. 
"""
function taylorprod(a,b)#función que devuelve el producto de los coeficientes tipo Taylor
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.n==y.n
        o=taylor(x.v,length(x.v)+5)
        f=taylor(y.v,length(x.v)+5)
        return Taylor(producto(o.v,f.v))
    elseif x.n<y.n
        z=taylor(x.v,y.n)
        return Taylor(producto(z.v,y.v))
    else
        z=taylor(y.v,x.n)
        return Taylor(producto(z.v,x.v))
    end
end
"""
division(a,b)
Función que regresa la división de dos vectores a,b de igual magnitud. Implementa las operaciones aritméticas de 
Taylor.
"""
function division(x,y)#división de los coeficientes tipo Taylor para dos vectores de igual número de elementos 
    h=1
    for m in 1:length(x)
        if(y[m]==0 && x[m]==0)
            h=m+1
        else
            break
        end
    end
    a=[x[i] for i=h:length(x)]
    b=[y[i] for i=h:length(x)]
    divab=[zero(eltype(x)) for i=h:length(x)]
    if b[1]!=zero(eltype(y))
        divab[1]=a[1]/b[1]
        for k in 2:length(a)
            l=zero(eltype(a))
            for i in 1:k
                l+=divab[i]*b[k-i+1]
            end
            divab[k]=(1/b[1])*(a[k]-l)
        end
        return divab
    else
        return error("b[h] debe ser distinto de cero")
    end
end
"""
taylordiv(a,b)
Función que regresa el división de dos estructuras tipo Taylor. 
"""
function taylordiv(a,b)#función que devuelve la división de los coeficientes tipo Taylor
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.n==y.n
        return Taylor(division(x.v,y.v))
    elseif x.n<y.n
        z=taylor(x.v,y.n)
        return Taylor(division(z.v,y.v))
    else
        z=taylor(y.v,x.n)
        return Taylor(division(x.v,z.v))
    end
end
"""
taylorigual(a,b)
Función que compara dos estructuras tipo Taylor. 
"""
function taylorigual(a,b)
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.v==y.v
        return true
    else
        return false
    end
end

taylorigual

In [3]:
import Base:+,-,*,/,==#falta -polinomio
for (f,f1,f2,f3) = ((:+,Taylor,Taylor,(:(taylorsum(a,b)))),
                    (:+,Taylor,Number,(:(Taylor(a)+Taylor(b)))),
                    (:+,Number,Taylor,(:(Taylor(a)+Taylor(b)))),
                    (:-,Taylor,Taylor,(:(taylorres(a,b)))),
                    (:-,Taylor,Number,(:(Taylor(a)-Taylor(b)))),
                    (:-,Number,Taylor,(:(Taylor(a)-Taylor(b)))),
                    (:*,Taylor,Taylor,(:(taylorprod(a,b)))),
                    (:*,Taylor,Number,(:(Taylor(a)*Taylor(b)))),
                    (:*,Number,Taylor,(:(Taylor(a)*Taylor(b)))),
                    (:/,Taylor,Taylor,(:(taylordiv(a,b)))),
                    (:/,Taylor,Number,(:(Taylor(a)/Taylor(b)))),
                    (:/,Number,Taylor,(:(Taylor(a)/Taylor(b)))),
                    (:(==),Taylor,Taylor,(:(taylorigual(a,b)))),
                    (:(==),Taylor,Number,(:(a==Taylor(b)))),
                    (:(==),Number,Taylor,(:(Taylor(a)==b))) )
              
    ex = quote
        $f(a::$f1,b::$f2)=$f3
    end
    @eval $ex
end
-(a::Taylor)=-1*a

- (generic function with 194 methods)

In [6]:
x=Taylor([1.0,1,1,1],7)
y=Taylor([1,1.0,1,1,0,0,0],7)
z=Taylor(7.0)
a=Taylor([1,1.,1,1,0,0,1],9)
b=Taylor([1,1.,0,0,0,0,1],2)
α=Taylor([0,0,1])
β=Taylor([0,1])
using Base.Test
@testset "pruebas con operaciones entre tipos Taylor"begin
    @test z==Taylor([7],0)
    @test x==Taylor([1,1,1,1],3)
    @test y==Taylor([1,1,1,1],3)
    @test z==7
    @test x+y==Taylor([2,2,2,2],3)
    @test 1+y==Taylor([2,1,1,1],3)
    @test y+1==Taylor([2,1,1,1],3)
    @test x-y==Taylor([0],0)
    @test -y==Taylor([-1,-1,-1,-1],3)
    @test -x==Taylor([-1,-1,-1,-1],3)
    @test y-x==Taylor([0],0)
    @test 1-y==Taylor([0,-1,-1,-1],3)
    @test y-1==Taylor([0,1,1,1],3)
    @test y*x==Taylor(producto([1.0,1,1,1,0,0,0],[1.0,1,1,1,0,0,0]))
    @test 2*y==Taylor([2,2,2,2],3)
    @test y*3==Taylor([3,3,3,3],3)
    @test 2/y==Taylor(division([2.0,0,0,0],[1.0,1,1,1]))
    @test y*(1/3)==Taylor([1/3,1/3,1/3,1/3],3)
    @test y/3==Taylor([1/3,1/3,1/3,1/3],3)
    @test α/β==Taylor([0,1])
    @test z/z==Taylor([1])
    @test x/x==Taylor([1])
end

Test Summary:                              | Pass  Total
  pruebas con operaciones entre tipos Taylor |   22     22


Base.Test.DefaultTestSet("pruebas con operaciones entre tipos Taylor",Any[],22,false)

In [7]:
workspace()
__precompile__(true)

module ATaylor
    import Base: +, -, *, /, ==
    export Taylor,taylor,producto,division,taylorsum,taylorres,taylorprod,taylordiv,taylorigual
"""
Taylor

Definición de polinomios de Taylor, donde v representa los coeficientes 
\$f_{[k]} = \\frac{f^{(k)}(x_0)}{k!} = \\frac{1}{k!}\\frac{d^k f}{dx^k}(x_0)\$ evaluados en \$x_0\$ y n es 
el grado del polinomio.
"""
type Taylor{T<:Number}
    v::Array{T,1}
    n::Int
end
function Taylor{T<:Number}(v::T)#función Taylor aplicada sobre un número
    x=[zero(eltype(v))]
    x[1]=v
    return Taylor(x,0)#devuelve una estructura Taylor de orden 0
end
function Taylor{T<:Number}(v::Array{T,1})#función Taylor aplicada sobre un vector
    i=length(v)#número de elementos del vector v
    while(v[i]==0&& i>1)
        i=i-1
    end
    x=zeros(eltype(v), i)#vector con i elementos
    for j in 1:i
        x[j]=v[j]
    end
    return Taylor(x,i-1)#regresa un Taylor con el orden mayor del vector v
end
"""
taylor(v,n)
Función que representa una estructura Taylor con n+1 elementos de v. Si los elementos de v son menores 
a n llena los elementos faltantes con ceros. Siempre regresa el orden del vector v.
"""
function taylor(v,n)#función Taylor aplicada sobre un vector
    x=Taylor(v)
    a=zeros(eltype(v), n+1)
    if x.n==n
        return x
    elseif x.n<n
        for i in 1:x.n+1
            a[i]=x.v[i]
        end
        return Taylor(a,x.n)
    else
        for i in 1:n+1
            a[i]=x.v[i]
        end
        return Taylor(a,x.n)
    end
end
"""
taylorsum(a,b)
Función que suma dos estructuras Taylor a y b.
"""
function taylorsum(a,b)#función que suma los coeficientes tipo Taylor
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.n==y.n#comparo los ordenes de la estructura Taylor de a y b
        return Taylor(x.v+y.v)#suma los vectores y regresa un Taylor
    elseif x.n<y.n
        z=taylor(x.v,y.n)
        return Taylor(z.v+y.v)
    else 
        z=taylor(y.v,x.n)
        return Taylor(x.v+z.v)
    end
end
"""
taylorres(a,b)
Función que resta dos estructuras Taylor a y b.
"""
function taylorres(a,b)#función que suma los coeficientes tipo Taylor
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.n==y.n#comparo los ordenes de la estructura Taylor de a y b
        return Taylor(x.v-y.v)#resta los vectores y regresa un Taylor
    elseif x.n<y.n
        z=taylor(x.v,y.n)
        return Taylor(z.v-y.v)
    else 
        z=taylor(y.v,x.n)
        return Taylor(x.v-z.v)
    end
end
"""
producto(a,b)
Función que regresa el producto de dos vectores a,b de igual magnitud. Implementa las operaciones aritméticas de 
Taylor.
"""
function producto(a,b)#producto de los coeficientes tipo Taylor para dos vectores de igual número de elementos
    ab=zeros(eltype(a), length(a))#vector con los mismos elementos de a
    for k in 1:length(a)
        for i in 1:k
            ab[k]+=a[i]*b[k+1-i]
        end
    end
    return ab
end
"""
taylorprod(a,b)
Función que regresa el producto de dos estructuras tipo Taylor. 
"""
function taylorprod(a,b)#función que devuelve el producto de los coeficientes tipo Taylor
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.n==y.n
        o=taylor(x.v,length(x.v)+5)
        f=taylor(y.v,length(x.v)+5)
        return Taylor(producto(o.v,f.v))
    elseif x.n<y.n
        z=taylor(x.v,y.n)
        return Taylor(producto(z.v,y.v))
    else
        z=taylor(y.v,x.n)
        return Taylor(producto(z.v,x.v))
    end
end
"""
division(a,b)
Función que regresa la división de dos vectores a,b de igual magnitud. Implementa las operaciones aritméticas de 
Taylor.
"""
function division(x,y)#división de los coeficientes tipo Taylor para dos vectores de igual número de elementos 
    h=1
    for m in 1:length(x)
        if(y[m]==0 && x[m]==0)
            h=m+1
        else
            break
        end
    end
    a=[x[i] for i=h:length(x)]
    b=[y[i] for i=h:length(x)]
    divab=[zero(eltype(x)) for i=h:length(x)]
    if b[1]!=zero(eltype(y))
        divab[1]=a[1]/b[1]
        for k in 2:length(a)
            l=zero(eltype(a))
            for i in 1:k
                l+=divab[i]*b[k-i+1]
            end
            divab[k]=(1/b[1])*(a[k]-l)
        end
        return divab
    else
        return error("b[h] debe ser distinto de cero")
    end
end
"""
taylordiv(a,b)
Función que regresa el división de dos estructuras tipo Taylor. 
"""
function taylordiv(a,b)#función que devuelve la división de los coeficientes tipo Taylor
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.n==y.n
        return Taylor(division(x.v,y.v))
    elseif x.n<y.n
        z=taylor(x.v,y.n)
        return Taylor(division(z.v,y.v))
    else
        z=taylor(y.v,x.n)
        return Taylor(division(x.v,z.v))
    end
end
"""
taylorigual(a,b)
Función que compara dos estructuras tipo Taylor. 
"""
function taylorigual(a,b)
    (x,y)=(Taylor(a.v),Taylor(b.v))
    if x.v==y.v
        return true
    else
        return false
    end
end
for (f,f1,f2,f3) = ((:+,Taylor,Taylor,(:(taylorsum(a,b)))),
                    (:+,Taylor,Number,(:(Taylor(a)+Taylor(b)))),
                    (:+,Number,Taylor,(:(Taylor(a)+Taylor(b)))),
                    (:-,Taylor,Taylor,(:(taylorres(a,b)))),
                    (:-,Taylor,Number,(:(Taylor(a)-Taylor(b)))),
                    (:-,Number,Taylor,(:(Taylor(a)-Taylor(b)))),
                    (:*,Taylor,Taylor,(:(taylorprod(a,b)))),
                    (:*,Taylor,Number,(:(Taylor(a)*Taylor(b)))),
                    (:*,Number,Taylor,(:(Taylor(a)*Taylor(b)))),
                    (:/,Taylor,Taylor,(:(taylordiv(a,b)))),
                    (:/,Taylor,Number,(:(Taylor(a)/Taylor(b)))),
                    (:/,Number,Taylor,(:(Taylor(a)/Taylor(b)))),
                    (:(==),Taylor,Taylor,(:(taylorigual(a,b)))),
                    (:(==),Taylor,Number,(:(a==Taylor(b)))),
                    (:(==),Number,Taylor,(:(Taylor(a)==b))) )
              
    ex = quote
        $f(a::$f1,b::$f2)=$f3
    end
    @eval $ex
end
-(a::Taylor)=-1*a

end

ATaylor

In [8]:
using Base.Test
using ATaylor

In [9]:
x=Taylor([1.0,1,1,1],7)
y=Taylor([1,1.0,1,1,0,0,0],7)
z=Taylor(7.0)
a=Taylor([1,1.,1,1,0,0,1],9)
b=Taylor([1,1.,0,0,0,0,1],2)
α=Taylor([0,0,1])
β=Taylor([0,1])
@testset "pruebas con operaciones entre tipos Taylor"begin
    @test z==Taylor([7],0)
    @test x==Taylor([1,1,1,1],3)
    @test y==Taylor([1,1,1,1],3)
    @test z==7
    @test x+y==Taylor([2,2,2,2],3)
    @test 1+y==Taylor([2,1,1,1],3)
    @test y+1==Taylor([2,1,1,1],3)
    @test x-y==Taylor([0],0)
    @test -y==Taylor([-1,-1,-1,-1],3)
    @test -x==Taylor([-1,-1,-1,-1],3)
    @test y-x==Taylor([0],0)
    @test 1-y==Taylor([0,-1,-1,-1],3)
    @test y-1==Taylor([0,1,1,1],3)
    @test y*x==Taylor(producto([1.0,1,1,1,0,0,0],[1.0,1,1,1,0,0,0]))
    @test 2*y==Taylor([2,2,2,2],3)
    @test y*3==Taylor([3,3,3,3],3)
    @test 2/y==Taylor(division([2.0,0,0,0],[1.0,1,1,1]))
    @test y*(1/3)==Taylor([1/3,1/3,1/3,1/3],3)
    @test y/3==Taylor([1/3,1/3,1/3,1/3],3)
    @test α/β==Taylor([0,1])
    @test z/z==Taylor([1])
    @test x/x==Taylor([1])
end

Test Summary:                              | Pass  Total
  pruebas con operaciones entre tipos Taylor |   22     22


Base.Test.DefaultTestSet("pruebas con operaciones entre tipos Taylor",Any[],22,false)

In [10]:
# Muestren que su código funciona con tests adecuados
workspace()
include("runtest_taylor.jl")

Test Summary:                              | Pass  Total
  pruebas con operaciones entre tipos Taylor |   22     22
Test Summary:                                           | Pass  Total
  pruebas con operaciones entre funciones de tipos Taylor |   13     13


Base.Test.DefaultTestSet("pruebas con operaciones entre funciones de tipos Taylor",Any[],13,false)

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.

**Para la función exponencial**

$$\displaystyle{E(x)=\exp(g(x))= \sum_{k=0}^\infty E_{[k]} (x-x_0)^k}\hspace{2cm}g(x)=\sum_{k=0}^\infty g_{[k]} (x-x_0)^k$$

$$\frac{dE(x)}{dx}=\sum_{k=0}^\infty kE_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dg(x)}{dx}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$
$$\frac{dE(x)}{dx}=\exp(g(x))\frac{dg(x)}{dx}$$

La relación de recurrencia es

\begin{equation}
E_{[k]} = \frac{1}{k} \sum_{j=0}^k j g_{[j]} \, E_{[k-j]} = 
\frac{1}{k} \sum_{j=0}^{k} (k-j) g_{[k-j]} \, E_{[j]}, \qquad k=1,2,\dots,
\end{equation}

In [11]:
workspace()
include("TaylorDiff.jl")
using ATaylor
"""
Cexp(g)
Función que regresa los coeficientes Taylor para la exp(g), donde g es un polinomio. 
"""
function Cexp(g)#algoritmo que cálcula los coeficientes de Taylor para la función exp(g(x))
    E=zeros(eltype(g.v), length(g.v))
    E[1]=exp(g.v[1])
    E[2]=g.v[2]*E[1]
    for k in 2:length(g.v)-1
        p=zero(eltype(g.v))
        for j in 0:k-1
            p+=(k-j)*g.v[k+1-j]*E[j+1]
        end
    E[k+1]=p/k
    end
    return Taylor(E)
end
g=taylor([0,1.],7)#Taylor que representa la función g(x)=x con un vector de 8 elementos
Cexp(g)

ATaylor.Taylor{Float64}([1.0,1.0,0.5,0.166667,0.0416667,0.00833333,0.00138889,0.000198413],7)

In [12]:
import Base:exp
exp(a::Taylor)=Cexp(a)

exp (generic function with 13 methods)

In [13]:
g=taylor([0,1.],7)
exp(g)

ATaylor.Taylor{Float64}([1.0,1.0,0.5,0.166667,0.0416667,0.00833333,0.00138889,0.000198413],7)

**Para el logaritmo** 

$$\displaystyle{L(x)=\log(g(x))= \sum_{k=0}^\infty L_{[k]} (x-x_0)^k}\hspace{2cm}g(x)=\sum_{k=0}^\infty g_{[k]} (x-x_0)^k$$

$$\frac{dL(x)}{dx}=\sum_{k=0}^\infty kL_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dg(x)}{dx}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$
$$\frac{dL(x)}{dx}=\frac{d\log(g(x))}{dx}=\frac{1}{g(x)}\frac{dg(x)}{dx}$$

$$g(x)\frac{dL(x)}{dx}=\frac{dg(x)}{dx}$$

Entonces
$$\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\sum_{j=0}^\infty jL_{[j]} (x-x_0)^{j-1}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\left[\sum_{k=0}^\infty kL_{[k]} (x-x_0)^{k-1}\right]\left[\sum_{j=0}^\infty g_{[j]} (x-x_0)^j\right]=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{k=1}^\infty\left[\sum_{j=0}^k(k-j)L_{[k-j]}g_{[j]} \right] (x-x_0)^{k-1}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{k=1}^\infty\left[\sum_{j=0}^kjL_{[j]}g_{[k-j]} \right] (x-x_0)^{k-1}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{j=0}^kjL_{[j]}g_{[k-j]}=kg_{[k]}$$

$$kL_{[k]}g_{[0]}+\sum_{j=0}^{k-1}jL_{[j]}g_{[k-j]}=kg_{[k]}$$

$$L_{[k]}=\frac{1}{g_{[0]}}\left[g_{[k]}-\frac{1}{k}\sum_{j=1}^{k-1}jL_{[j]}g_{[k-j]}\right]$$

In [14]:
"""
coeflog(g)
Función que regresa los coeficientes Taylor para el ln(g), donde g es un polinomio. 
"""
function coeflog(g)#algoritmo que cálcula los coeficientes de Taylor para la función log(g(x))
    L=zeros(eltype(g.v), length(g.v))
    L[1]=log(g.v[1])
    L[2]=g.v[2]/g.v[1]
    for k in 2:length(g.v)-1
    l=zero(eltype(g.v))
        for j in 1:k-1
            l+=j*L[j+1]*g.v[k+1-j]
        end
    L[k+1]=(1/g.v[1])*(g.v[k+1]-l/k)
    end
    return Taylor(L)
end
g=taylor([1,1//1],7)#Taylor que representa la función g(x)=1+x con un vector de 8 elementos 
coeflog(g)

ATaylor.Taylor{Rational{Int64}}(Rational{Int64}[0//1,1//1,-1//2,1//3,-1//4,1//5,-1//6,1//7],7)

In [15]:
import Base:log
log(a::Taylor)=coeflog(a)

log (generic function with 20 methods)

In [16]:
g=taylor([1,1//1],7)#Taylor que representa la función g(x)=1+x con un vector de 8 elementos 
log(g)

ATaylor.Taylor{Rational{Int64}}(Rational{Int64}[0//1,1//1,-1//2,1//3,-1//4,1//5,-1//6,1//7],7)

**Para la función $P_\alpha(x)=(g(x))^\alpha$** 

$$\displaystyle{P_\alpha(x)=(g(x))^\alpha= \sum_{k=0}^\infty {P_\alpha}_{[k]} (x-x_0)^k}\hspace{2cm}g(x)=\sum_{k=0}^\infty g_{[k]} (x-x_0)^k$$

$$\frac{dP_\alpha(x)}{dx}=\sum_{k=0}^\infty k{P_\alpha}_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dg(x)}{dx}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$
$$\frac{d(g(x))^\alpha}{dx}=\alpha (g(x))^{\alpha-1}\frac{dg(x)}{dx}=\alpha \frac{(g(x))^{\alpha}}{g(x)}\frac{dg(x)}{dx}$$

Por una parte
$$g(x)\frac{d(g(x))^\alpha}{dx}=\alpha(g(x))^{\alpha}\frac{dg(x)}{dx}$$

$$\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\sum_{j=0}^\infty j{P_\alpha}_{[j]} (x-x_0)^{j-1}=
\alpha \sum_{j=0}^\infty {P_\alpha}_{[j]} (x-x_0)^j\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{k=1}^\infty\left[\sum_{j=0}^kg_{[k-j]}j{P_\alpha}_{[j]}\right](x-x_0)^{k-1}=
\sum_{k=1}^\infty\left[\sum_{j=0}^k \alpha(k-j)g_{[k-j]}{P_\alpha}_{[j]}\right](x-x_0)^{k-1}$$

$$\sum_{j=0}^kg_{[k-j]}j{P_\alpha}_{[j]}=\sum_{j=0}^k \alpha(k-j)g_{[k-j]}{P_\alpha}_{[j]}$$

$$\sum_{j=1}^kg_{[k-j]}j{P_\alpha}_{[j]}=\sum_{j=0}^{k-1} \alpha(k-j)g_{[k-j]}{P_\alpha}_{[j]}$$

$$g_{[0]}k{P_\alpha}_{[k]}+\sum_{j=1}^{k-1}g_{[k-j]}j{P_\alpha}_{[j]}=\alpha kg_{[k]}{P_\alpha}_{[0]}+\sum_{j=1}^{k-1} \alpha(k-j)g_{[k-j]}{P_\alpha}_{[j]}$$

$$g_{[0]}k{P_\alpha}_{[k]}=\alpha kg_{[k]}{P_\alpha}_{[0]}+\sum_{j=1}^{k-1}g_{[k-j]}{P_\alpha}_{[j]}\left[\alpha(k-j)-j\right]$$

$${P_\alpha}_{[k]}=\alpha \frac{g_{[k]}}{g_{[0]}}{P_\alpha}_{[0]}+\frac{1}{kg_{[0]}}\sum_{j=1}^{k-1}g_{[k-j]}{P_\alpha}_{[j]}\left[\alpha k-(\alpha +1)j\right]$$

In [17]:
"""
Cexpalpha(g,alpha)
Función que regresa los coeficientes Taylor para (g)^α, donde g es un polinomio. 
"""
function Cexpalpha(g,alpha)#algoritmo que cálcula los coeficientes de Taylor para la función (g(x))^α
    P=zeros(eltype(g.v), length(g.v))
    P[1]=(g.v[1])^alpha
    P[2]=alpha*(g.v[2]/g.v[1])*P[1]
    for k in 2:length(g.v)-1
    p=zero(eltype(g.v))
        for j in 1:k-1
            p+=(alpha*k-(alpha+1)*j)*g.v[k+1-j]*P[j+1]
        end
    P[k+1]=alpha*(g.v[k+1]/g.v[1])*P[1]+(1/(k*g.v[1]))*p
    end
    return Taylor(P)
end
g=taylor([1,1.],7)#Taylor que representa la función g(x)=1+x con un vector de 8 elementos 
Cexpalpha(g,4)

ATaylor.Taylor{Float64}([1.0,4.0,6.0,4.0,1.0],4)

In [18]:
import Base:^
^(a::Taylor,b::Number)=Cexpalpha(a,b)

^ (generic function with 49 methods)

In [19]:
g=taylor([1,1.],7)#Taylor que representa la función g(x)=1+x con un vector de 8 elementos 
g^5.0

ATaylor.Taylor{Float64}([1.0,5.0,10.0,10.0,5.0,1.0],5)

**Para la función $S(x)=\sin(g(x))$ y $C(x) = \cos(g(x))$** 

$$S(x)=sin(g(x))= \sum_{k=0}^\infty {S}_{[k]} (x-x_0)^k\hspace{2cm}C(x) = \cos(g(x))=\sum_{k=0}^\infty C_{[k]} (x-x_0)^k$$

$$\frac{dS(x)}{dx}=\sum_{k=0}^\infty k{S}_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dC(x)}{dx}=\sum_{k=0}^\infty k{C}_{[k]} (x-x_0)^{k-1}$$
$$\frac{dS(x)}{dx}=\cos(g(x))\frac{dg(x)}{dx}=C(x)\frac{dg(x)}{dx}\hspace{2cm}
\frac{dC(x)}{dx}=-\sin(g(x))\frac{dg(x)}{dx}=-S(x)\frac{dg(x)}{dx}$$

Igualando las últimas 4 expresiones
$$\frac{dS(x)}{dx}=\sum_{k=0}^\infty k{S}_{[k]} (x-x_0)^{k-1}=
\sum_{k=0}^\infty C_{[k]} (x-x_0)^k\sum_{j=0}^\infty jg_{[j]} (x-x_0)^{j-1}=
\sum_{k=1}^\infty \left[\sum_{j=0}^k jg_{[j]}C_{[k-j]}\right](x-x_0)^{k-1}
$$
$$\frac{dC(x)}{dx}=\sum_{k=0}^\infty k{C}_{[k]} (x-x_0)^{k-1}=
-\sum_{k=0}^\infty S_{[k]} (x-x_0)^k\sum_{j=0}^\infty jg_{[j]} (x-x_0)^{j-1}=
-\sum_{k=1}^\infty \left[\sum_{j=0}^k jg_{[j]}S_{[k-j]}\right](x-x_0)^{k-1}
$$

De aquí obtengo las dos siguientes relaciones

$$S_{[k]}=\frac{1}{k}\sum_{j=0}^k jg_{[j]}C_{[k-j]}=\frac{1}{k}\sum_{j=0}^{k-1} (k-j)g_{[k-j]}C_{[j]}\hspace{2cm}
C_{[k]}=-\frac{1}{k}\sum_{j=0}^k jg_{[j]}S_{[k-j]}=-\frac{1}{k}\sum_{j=0}^{k-1} (k-j)g_{[k-j]}S_{[j]}$$

Con las condiciones iniciales 
$$S_{[0]}=\sin(g(x_0))\hspace{2cm}C_{[0]}=\cos(g(x_0))$$

Para el caso concreto $g(x) = x$ alrededor de $x_0=0$, donde tenemos $g_{[j]} = \delta_{j,1}$, obtenemos

$$S_{[0]}=0 \hspace{2cm}C_{[0]}=1$$
$$S_{[1]}=C_{[0]}=1 \hspace{2cm}C_{[1]}=-S_{[0]}=0$$

$$S_{[k]}=\frac{1}{k}C_{[k-1]}=\frac{1(-1)}{k(k-1)}S_{[k-2]}=\frac{1(-1)(1)}{k(k-1)(k-2)}C_{[k-3]}=
\frac{1(-1)(1)(-1)}{k(k-1)(k-2)(k-3)}S_{[k-4]}=\dots$$$$
C_{[k]}=\frac{-1}{k}S_{[k-1]}=\frac{(-1)(1)}{k(k-1)}C_{[k-2]}=\frac{(-1)(1)(-1)}{k(k-1)(k-2)}S_{[k-3]}=
\frac{(-1)(1)(-1)(1)}{k(k-1)(k-2)(k-3)}C_{[k-4]}=\dots$$

$$S_{[k]}=\frac{(-1)}{k(k-1)}S_{[k-2]}=\frac{(-1)^2}{k(k-1)(k-2)(k-3)}S_{[k-4]}=
\frac{(-1)^3}{k(k-1)(k-2)(k-3)(k-4)(k-5)}S_{[k-6]}=\dots$$$$
C_{[k]}=\frac{(-1)}{k(k-1)}C_{[k-2]}=\frac{(-1)^2}{k(k-1)(k-2)(k-3)}C_{[k-4]}=
\frac{(-1)^3}{k(k-1)(k-2)(k-3)(k-4)(k-5)}C_{[k-6]}=\dots$$

$$S_{[k]}=\frac{(-1)^{j/2}(k-j)!}{k!}S_{[k-j]}\hspace{2cm}C_{[k]}=\frac{(-1)^{j/2}(k-j)!}{k!}C_{[k-j]}$$

Para que $S_{[k]}$ no sea nulo $k>2$ y debe ser un número impar $k=2n+1$. De la misma menera para que $C_{[k]}$ no sea nulo $k\geq2$ y debe ser un número par $k=2n$. Entonces

$$S_{[2n+1]}=\frac{(-1)^{j/2}((2n+1)-j)!}{(2n+1)!}S_{[(2n+1)-j]}\hspace{2cm}C_{[2n]}=\frac{(-1)^{j/2}((2n)-j)!}{(2n)!}C_{[(2n)-j]}$$

Para los coeficientes del seno si $2n+1-j=1$ $\Rightarrow j=2n$. En los coeficientes del coseno si $2n-j=0$ $\Rightarrow j=2n$. De esta manera

$$S_{[2n+1]}=\frac{(-1)^{n}}{(2n+1)!}S_{[1]}=\frac{(-1)^{n}}{(2n+1)!}C_{[0]}=\frac{(-1)^{n}}{(2n+1)!}\hspace{2cm}C_{[2n]}=\frac{(-1)^{n}}{(2n)!}C_{[0]}=\frac{(-1)^{n}}{(2n)!}$$

Por lo tanto
$$S(x)=\sin(x)=\sum_{n=0}^\infty \frac{(-1)^{n}}{(2n+1)!}x^{2n+1}\hspace{2cm}C(x) = \cos(x)=\sum_{n=0}^\infty \frac{(-1)^{n}}{(2n)!}x^{2n}$$

In [20]:
"""
Csencos(g)
Función que regresa a la vez,los coeficientes Taylor para sin(g) y cos(g), donde g es un polinomio. 
"""
function Csencos(g)#algoritmo que cálcula los coeficientes de Taylor para las funciones cos(g(x)) y sin(g(x))
    S=zeros(eltype(g.v), length(g.v))
    C=zeros(eltype(g.v), length(g.v))
    C[1]=cos(g.v[1])
    S[2]=C[1]
    for k in 2:length(g.v)-1
        p=zero(eltype(g.v))
        q=zero(eltype(g.v))
        for j in 0:k-1
            p+=(k-j)*g.v[k+1-j]*C[j+1]
            q+=(k-j)*g.v[k+1-j]*S[j+1]
        end
    S[k+1]=p/k
    C[k+1]=-q/k
    end
    return Taylor(C),Taylor(S)
end
"""
Csen(g)
Función que regresa los coeficientes Taylor para sin(g), donde g es un polinomio. 
"""
function Csen(g)
    (a,b)=Csencos(g)
    return b
end
"""
coeflog(g)
Función que regresa los coeficientes Taylor para cos(g), donde g es un polinomio. 
"""
function Ccos(g)
    (a,b)=Csencos(g)
    return a
end
g=taylor([0,1//1],7)#Taylor que representa la función g(x)=x con un vector de 8 elementos 
Ccos(g)

ATaylor.Taylor{Rational{Int64}}(Rational{Int64}[1//1,0//1,-1//2,0//1,1//24,0//1,-1//720],6)

In [21]:
import Base:sin,cos
sin(a::Taylor)=Csen(a)
cos(a::Taylor)=Ccos(a)

cos (generic function with 12 methods)

In [22]:
g=taylor([0,1//1],7)#Taylor que representa la función g(x)=x con un vector de 8 elementos
sin(g),cos(g)

(ATaylor.Taylor{Rational{Int64}}(Rational{Int64}[0//1,1//1,0//1,-1//6,0//1,1//120,0//1,-1//5040],7),ATaylor.Taylor{Rational{Int64}}(Rational{Int64}[1//1,0//1,-1//2,0//1,1//24,0//1,-1//720],6))

In [23]:
a=[0,1.]
b=[1,1.]
x=taylor(a,4)
y=taylor(b,4)

using Base.Test
@testset "pruebas con operaciones entre funciones de tipos Taylor"begin
    @test exp(x)==Taylor([1,1,1/2,1/6,1/24],4)
    @test exp(x)==Cexp(x)
    @test exp(taylor(a,3))==Taylor([1,1,1/2,1/6],3)
    @test log(y)==Taylor([0,1,-1/2,1/3,-1/4],4)
    @test log(y)==coeflog(taylor(b,4))
    @test y^5.0==Cexpalpha(y,5)
    @test y^3.0==Taylor([1.,3,3,1],3)
    @test sin(x)==Csen(x)
    @test sin(x)==Csen(taylor(a,4))
    @test cos(x)==Ccos(x)
    @test cos(x)==Ccos(taylor(a,4))
    @test y^2.0==Taylor(producto([1.0,1.0,0.0,0.0,0.0],[1.0,1.0,0.0,0.0,0.0]))
    @test y^2.0==y*y    
end

Test Summary:                                           | Pass  Total
  pruebas con operaciones entre funciones de tipos Taylor |   13     13


Base.Test.DefaultTestSet("pruebas con operaciones entre funciones de tipos Taylor",Any[],13,false)

**Aquí incluyo las pruebas finales.**

In [24]:
workspace()
include("runtest_taylor.jl")

Test Summary:                              | Pass  Total
  pruebas con operaciones entre tipos Taylor |   22     22
Test Summary:                                           | Pass  Total
  pruebas con operaciones entre funciones de tipos Taylor |   13     13


Base.Test.DefaultTestSet("pruebas con operaciones entre funciones de tipos Taylor",Any[],13,false)